In [36]:
import pandas as pd
df = pd.read_csv("/kaggle/input/quickans-answers/jaccard_scores.csv")
df.head()

,No.,Topic,Question,Ground Truth Answer,QuickAns answer,Jaccard Sim QA,GPT 3.5 answer,Jaccard Sim GPT,Our Comments
0,194.0,Requesting Clarification on Quiz 2 - Question ...,"The question was, V = w 1 , w 2 , w 3 , w 4 V...",The reasoning for this question is similar to ...,NaN,NaN,NaN,NaN,Varun: I don't think we can even expect this a...
1,25.0,NaN,How is Jelinek-Mercer smoothing different from...,"There is really no difference. Yes, JM is a pa...",Jelinek-Mercer smoothing is a specific type of...,0.143,Jelinek-Mercer smoothing and interpolation are...,0.100,NaN
2,39.0,Calculation of Average Precision,Can someone explain to me this calculation? I'...,Have you taken CS410? This is a basic concept ...,NaN,NaN,NaN,NaN,Prathamesh: need to add question
3,NaN,KL Divergence,Going back to the point that KL-divergence is ...,This is an excellent question! There are many ...,"Thank you for your question. Yes, there are ca...",0.072,"Yes, there are cases where unsymmetric measure...",0.056,NaN
4,NaN,MLE estimate of N-gram LM equation,does the order of the words in the denominator...,The following responses should have already ad...,NaN,NaN,NaN,NaN,Ishika: should we be able to handle this? The ...


In [37]:
quickans_null_map = df["QuickAns answer"].isnull()
chatgpt_null_map = df["GPT 3.5 answer"].isnull()
for index, row in df.iterrows():
    if quickans_null_map[index] or chatgpt_null_map[index]:
        df.drop(index, inplace=True)

In [51]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


def get_similarity_mpnet(sentence1, sentence2):
    encoded_input = tokenizer([sentence1, sentence2], padding=True, truncation=True, return_tensors='pt')

    # token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
#     print(sentence_embeddings[0].shape)
    cosine_sim = F.cosine_similarity(sentence_embeddings[0].unsqueeze(0), sentence_embeddings[1].unsqueeze(0), dim = 1)
    return float(cosine_sim[0])
print(get_similarity_mpnet("hello world", "hello world"))

1.0000001192092896


In [52]:
for index, row in df.iterrows():
    print(index)
    reference_text = row["Question"] + row["Ground Truth Answer"]
    sim_qa = get_similarity_mpnet(reference_text, row["QuickAns answer"])
    sim_chatgpt = get_similarity_mpnet(reference_text, row["GPT 3.5 answer"])
    print(index)
    df.loc[index, 'Sentence Sim QA'] = sim_qa
    df.loc[index, 'Sentence Sim ChatGPT'] = sim_chatgpt

1
1
3
3
5
5
6
6
8
8
9
9
10
10
11
11
12
12
13
13
14
14


In [53]:
df

,No.,Topic,Question,Ground Truth Answer,QuickAns answer,Jaccard Sim QA,GPT 3.5 answer,Jaccard Sim GPT,Our Comments,Sentence Sim QA,Sentence Sim ChatGPT
1,25.0,NaN,How is Jelinek-Mercer smoothing different from...,"There is really no difference. Yes, JM is a pa...",Jelinek-Mercer smoothing is a specific type of...,0.1430,Jelinek-Mercer smoothing and interpolation are...,0.1000,NaN,0.806562,0.705489
3,NaN,KL Divergence,Going back to the point that KL-divergence is ...,This is an excellent question! There are many ...,"Thank you for your question. Yes, there are ca...",0.0720,"Yes, there are cases where unsymmetric measure...",0.0560,NaN,0.865957,0.808657
5,NaN,Continuation of #52 MLE estimate of N-gram LM ...,In the following equation what if the denomina...,That's a good question. One solution is to do ...,Thank you for your question. In the equation y...,0.0690,"If the denominator is zero, then the probabili...",0.0560,Prathamesh: need to add question,0.660463,0.758888
6,75.0,Can blind feedback be applied repeatedly?,"Basically, use the original query model q0 to ...",Great question & great idea! The short answer ...,Based on the information provided in the cours...,0.1010,"Yes, blind feedback can be applied repeatedly....",0.0900,NaN,0.796495,0.853232
8,93.0,How to tell if a retrieval function has IDF we...,How do you tell if a retrieval function has ID...,Since IDF is primarily to formulate against th...,To determine if a retrieval function has IDF w...,0.0800,To determine if a retrieval function has IDF w...,0.0860,NaN,0.818190,0.940957
9,103.0,Divergence minimization question,"Hi, I was wondering about Divergence minimizat...","It's because in line 3, -lambda*H(theta;theta_...","Dear student, Thank you for your question reg...",0.0070,"As an AI language model, I do not have access ...",0.0350,NaN,0.787487,0.550954
10,105.0,Rank equivalence,Is there any difference between two quantities...,Perhaps some operations may not hold the propo...,Proportional and rank equivalent are two diffe...,0.0738,There is a difference between two quantities b...,0.1000,NaN,0.721358,0.923400
11,111.0,Assignment2 Question 2.3,Dirichlet prior smoothing. I can write the equ...,This is an open-ended question and it would in...,Thank you for your question. You are correct...,0.0640,The influence of k on each part of the equatio...,0.0450,Ishika: shouldn't expect this but will try it ...,0.847058,0.760666
12,120.0,A question about 2-stage smoothing,"In this picture, if w occurs very frequently i...","Here, if w is frequent in the background model...",Thank you for your question. Let me explain ho...,0.0800,"As an AI language model, I do not have access ...",0.0570,Prathamesh: need to add question,0.708458,0.628870
13,130.0,Log likelihood in EM algorithm,"Hello, this is a dumb question but I was wonde...","First of all, you need to understand that like...","Dear student, Thank you for your question. Th...",0.0802,"To calculate the log-likelihood value, you fir...",0.0540,Prathamesh: need to add question,0.809385,0.649986


In [54]:
import numpy as np
from scipy.stats import ttest_ind

def calculate_scores(score_name, set1_scores, set2_scores):
    if len(set1_scores) != len(set2_scores):
        print("Error: different number of scores in the two sets")
        exit()

    t_stat, p_value = ttest_ind(set1_scores, set2_scores)

    # Print the results
    print("Stats for {} Scores:".format(score_name))
    print("T-test statistic: ", t_stat)
    print("P-value: ", p_value)

In [55]:
# Sentence-Similarity Scores
set1_scores = np.array(df["Sentence Sim QA"])
set2_scores = np.array(df["Sentence Sim ChatGPT"])
calculate_scores("Sentence Similarity", set1_scores, set2_scores)

Stats for Sentence Similarity Scores:
T-test statistic:  0.6650148775877179
P-value:  0.5136371262660309


In [56]:
df.to_csv("mpnet_scores.csv")